# 导入工具包

In [1]:
import random
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
# from keras.layers import Lambda, Dense
from keras.layers import *
import tensorflow as tf

Using TensorFlow backend.


# 设置参数

In [2]:
set_gelu('tanh')  # 切换gelu版本

In [3]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = '../model/albert_small_zh_google/albert_model.ckpt'
dict_path = '../model/albert_small_zh_google/vocab.txt'

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 加载BERT模型

In [4]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

# 定义BiLSTM网络

## 获取bert的char embeeding

In [5]:
lstm_input = bert.layers['Embedding-Token'].output

In [6]:
X = Bidirectional(LSTM(128, return_sequences=True))(lstm_input)
lstm_output = Bidirectional(LSTM(128))(X)

## 分类全联接

In [7]:
output = Dense(
    units=num_classes,
    activation='softmax'
)(lstm_output)

## 定义模型输入输出

In [8]:
model = keras.models.Model(bert.model.input[0], output)

## 查看模型结构

In [9]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input-Token (InputLayer)     (None, None)              0         
_________________________________________________________________
Embedding-Token (Embedding)  (None, None, 128)         2704384   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 514       
Total params: 3,362,306
Trainable params: 3,362,306
Non-trainable params: 0
_________________________________________________________________


## 编译模型

In [10]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  
    metrics=['accuracy'],
)

# 生成数据

## 加载数据

In [11]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    df_train_data = pd.read_csv("../data/train.csv").\
    drop_duplicates(['level_1', 'level_2', 'level_3', 'level_4', 'content', 'label'])
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [12]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

## 迭代器生成

In [13]:
class data_generator(DataGenerator):
    def __iter__(self, random=False):
        batch_token_ids, batch_labels = [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids], batch_labels
                batch_token_ids, batch_labels = [], []

In [14]:
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

# 训练、验证和预测

In [15]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

In [16]:
class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

In [17]:
def data_pred(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        y_pred = int(model.predict([token_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

## 训练和验证模型

In [18]:
evaluator = Evaluator()

In [19]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=10,
        callbacks=[evaluator]
    )

/root/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/10
234/234 [==============================] - 171s 730ms/step - loss: 0.5201 - accuracy: 0.9257
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 2/10
234/234 [==============================] - 171s 731ms/step - loss: 0.2515 - accuracy: 0.9306
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 3/10
234/234 [==============================] - 167s 713ms/step - loss: 0.2434 - accuracy: 0.9306
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 4/10
234/234 [==============================] - 168s 717ms/step - loss: 0.2384 - accuracy: 0.9306
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 5/10
234/234 [==============================] - 170s 726ms/step - loss: 0.2329 - accuracy: 0.9306
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 6/10
234/234 [==============================] - 170s 728ms/step - loss: 0.2270 - accuracy: 0.9306
val_acc: 0.92192, best_val_acc: 0.92192, test_acc: 0.92192

Epoch 7/10
234/2

# 查看模型训练和验证结果

## 加载最好的模型

In [20]:
model.load_weights('best_model.weights')

## 验证集结果

In [21]:
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.932546



## 训练集结果

In [22]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.926101



# 模型预测保存结果

In [ ]:
id_ids, y_pred_ids = data_pred(test_data)
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

df_save.to_csv('result.csv')